In [ ]:
import os
import matplotlib
import plotly
import plotly.express as px
import pandas as pd

In [ ]:
DOE_NAME = "doe2"

df_res = pd.read_csv(DOE_NAME + "_res.csv")  # doe1_res.csv
df_res.sort_values(by="FIT", inplace=True, ascending=False)
df_res["RMSE"] = df_res["RMSE"].fillna(1000)
df_res["FIT"] = df_res["FIT"].fillna(0.0)
df_res["FIT"] = df_res["FIT"] * (df_res["FIT"] > 0) # minimum fit to 0% (easier to interpret)


In [ ]:
factors = ["max_time", "seq_len", "est_frac", "est_direction", "est_type", "est_hidden_size"]
for factor in factors:
    df_res[factor] = df_res[factor].astype("category")

In [ ]:
df_res.head(10)

In [ ]:
df_res.tail(10)

In [ ]:
fig = px.histogram(df_res, x="FIT", facet_col="est_type", facet_row="est_direction", color="seq_len")
fig.show()

In [ ]:
fig = px.scatter(df_res, y="FIT",
                 facet_col="est_type", facet_row="est_direction", color="seq_len",
                 hover_data=["est_frac", "max_time", "est_hidden_size"])
fig.show()

In [ ]:
df_good = df_res[(df_res["seq_len"] != 40) & (df_res["est_direction"] == "forward")]

In [ ]:
fig = px.scatter(df_good, y="FIT", x= "seq_len", color="est_hidden_size",
                 facet_col="est_type", facet_row="max_time",
                 hover_data=["est_frac", "max_time", "est_hidden_size"])
fig.show()

In [ ]:
fig = px.scatter(df_res, y="FIT", x="est_type")
fig.show()

In [ ]:
df_5min = df_res[df_res["max_time"] == 300]

In [ ]:
fig = px.scatter(df_5min, y="FIT",
                 facet_col="est_type", facet_row="est_direction", color="seq_len",
                 hover_data=["est_frac", "max_time", "est_hidden_size"])
fig.show()